<h1> GENRE: RAP </h1>

<h3>(1) Access the data from the dataset </h3>


In [1]:
%%capture
%pip install datasets

from datasets import load_dataset

ds = load_dataset("sebastiandizon/genius-song-lyrics")

In [36]:
# Assuming the dataset has a 'category' column
unique_categories = ds['train'].unique('tag')  # Get all unique categories

# Group into subdatasets
subdatasets = {category: ds['train'].filter(lambda x: x['tag'] == category and x['language'] == 'en') for category in unique_categories}

type_of_music = 'rap'
# Group into music decades
#subdataset_70_without_sort = ds['train'].filter(lambda song: 1960 <= song["year"] <= 1970 and song['tag'] == type_of_music)
subdataset_70 = ds['train'].filter(lambda song: 1960 <= song["year"] <= 1970 and song['tag'] == type_of_music).sort('views', reverse=True)
subdataset_80 = ds['train'].filter(lambda song: 1970 <= song["year"] <= 1980 and song['tag'] == type_of_music).sort('views', reverse=True)
subdataset_90 = ds['train'].filter(lambda song: 1980 <= song["year"] <= 1990 and song['tag'] == type_of_music).sort('views', reverse=True)
subdataset_00 = ds['train'].filter(lambda song: 1990 <= song["year"] <= 2000 and song['tag'] == type_of_music).sort('views', reverse=True)
subdataset_10 = ds['train'].filter(lambda song: 2000 <= song["year"] <= 2010 and song['tag'] == type_of_music).sort('views', reverse=True)
subdataset_20 = ds['train'].filter(lambda song: 2010 <= song["year"] <= 2020 and song['tag'] == type_of_music).sort('views', reverse=True)
subdataset_30 = ds['train'].filter(lambda song: 2020 <= song["year"] <= 2030 and song['tag'] == type_of_music).sort('views', reverse=True)

print("Done")

# Create subdataset
#eng_rap = subdatasets['rap']

done
[2360514, 1625352, 807531, 691578, 649119]


<h2> LDA </h2>

<h2> Remove the bad words</h2>

In [6]:
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("tushifire/ldnoobw")
path = kagglehub.dataset_download("sahib12/badwords")

#file = kagglehub.load_dataset("tushifire/ldnoobw",path)

print("Path to dataset files:", path)

swear_words_file = os.path.join(path, 'Terms-to-Block.csv')  # Replace 'en.txt' with the correct filename

with open(swear_words_file, 'r') as f:
    swear_words = set(f.read().splitlines())

print(f"!!!!Loaded {len(swear_words)} swear words.")

#print(swear_words)

Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.


In [18]:
import gensim
import re
from gensim.corpora.dictionary import Dictionary
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from gensim.parsing.preprocessing import remove_stopwords

# Sicherstellen, dass NLTK-Daten heruntergeladen sind
nltk.download('stopwords')
nltk.download('punkt')

# Beispiel-Liste von Strings


# Stopwords definieren
stop_words = set(stopwords.words('english')+["like", "oh", "na", "la", "yo", "get"])

# Preprocessing: Tokenisieren, Stopwords entfernen, Kleinbuchstaben
def preprocess(doc):
    doc = re.sub(r"[^a-zA-Z0-9\s]", " ", doc) # delete unncessary things
    #doc = remove_stopwords(doc.lower())
    doc = remove_repeated_lines(doc)
    
    tokens = word_tokenize(doc.lower())  # Kleinbuchstaben und Tokenisierung
    
    tokens = [word for word in tokens if word not in stop_words]  # Stopwords entfernen
    
    return tokens

def remove_repeated_lines(doc):
    lines = doc.split("\n")
    return "\n".join(sorted(set(lines), key=lines.index))

def sort_out_bad_words(doc,word_list, replacement="swear"):
    """
    Ersetzt Wörter in einem Text, wenn sie in einer Liste vorkommen.
    
    :param text: Der Eingabetext (String).
    :param word_list: Liste der Wörter, die ersetzt werden sollen.
    :param replacement: Der Text, durch den die Wörter ersetzt werden sollen.
    :return: Der bearbeitete Text.
    """
    # Erstelle ein Set für einen schnelleren Lookup
    word_set = set(word_list)
    
    # Splitte den Text in Wörter
    words = doc.split()
    
    # Ersetze jedes Wort, falls es in der Liste vorkommt
    replaced_words = [replacement if word.lower() in word_set else word for word in words]
    
    # Füge den Text wieder zusammen
    return " ".join(replaced_words)
 

def process_subdatasets(subdataset):
    # Get rid of the bad words and replace it with "swear word"
    without_swear = [sort_out_bad_words(doc, swear_words ) for doc in subdataset]

    # Preprocessing für alle Dokumente anwenden
    processed_docs = [preprocess(doc) for doc in without_swear]

    # Erstellen eines Wörterbuchs
    dictionary = Dictionary(processed_docs)

    # Erstellen eines Bag-of-Words-Korpus
    corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

    # LDA-Modell trainieren
    lda_model = gensim.models.LdaModel(
        corpus=corpus,
        id2word=dictionary,
        num_topics=5,  # Anzahl der Themen
        random_state=42,
        passes=40,  # Anzahl der Durchgänge
        alpha='symmetric',
        eta='auto'
        #alpha='auto'  # Dirichlet-Hyperparameter Entscheidung zwischen Auto und symetric
    )
    
    id2word = dictionary
    dict_topic = {}
    # Themen anzeigen
    #print("Themen und ihre Schlüsselwörter:")
    for idx, topic in lda_model.print_topics(num_words=5):
        #print(f"Topic {idx}: {topic}")
        dict_topic[idx] = topic 

    return dict_topic


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/maurice/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/maurice/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
amount = 100


rap_70_lyrics = process_subdatasets(subdataset_70['lyrics'][:amount])
rap_80_lyrics = process_subdatasets(subdataset_80['lyrics'][:amount])
rap_90_lyrics = process_subdatasets(subdataset_90['lyrics'][:amount])
rap_00_lyrics = process_subdatasets(subdataset_00['lyrics'][:amount])
rap_10_lyrics = process_subdatasets(subdataset_10['lyrics'][:amount])
rap_20_lyrics = process_subdatasets(subdataset_20['lyrics'][:amount])
rap_30_lyrics = process_subdatasets(subdataset_30['lyrics'][:amount])

#list_of_lyrics = [rap_70_lyrics,rap_80_lyrics]
list_of_lyrics = {
    "70s": rap_70_lyrics,
    "80s": rap_80_lyrics,
    "90s": rap_90_lyrics,
    "00s": rap_00_lyrics,
    "10s": rap_10_lyrics,
    "20s": rap_20_lyrics,
    "30s": rap_30_lyrics
}

import csv

data_for_csv = []

for decade_name, decade in list_of_lyrics.items():
    i = 0
    for topics in decade:
        i = i + 1
        data_for_csv.append({
            "Method": "LDA",
            "Genre": "Rap",
            "Decade": decade_name,
            "Topic": str(decade[topics])
        })


# Datei schreiben
output_file = "Analysis_Folder/topics_rap_LDA.csv"
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["Method", "Genre", "Decade", "Topic"])
    writer.writeheader()  # Schreibe die Kopfzeile
    writer.writerows(data_for_csv)  # Schreibe die Daten

print(f"CSV-Datei '{output_file}' wurde erfolgreich erstellt!")


CSV-Datei 'Analysis_Folder/topics_rap_LDA.csv' wurde erfolgreich erstellt!


In [6]:
type(rap_70_lyrics[0])

str

# tipps von ChaGPT
https://chatgpt.com/c/676f1ffc-b134-8012-9f03-6bd466bda148

- Herausfinden, wieviele Topics am geeingesten sind
- warum sind weiterhin so viele Stop words enthalten
- wieviele Lieder nehm ich überhaupt

In [7]:
print(len(subdataset_80))

440


In [8]:
print(len(subdataset_90))

4505


<h1> BERT TOPIC MODELLING </h1>


In [11]:
import gensim
import re
from gensim.corpora.dictionary import Dictionary
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from gensim.parsing.preprocessing import remove_stopwords

# Sicherstellen, dass NLTK-Daten heruntergeladen sind
nltk.download('stopwords')
nltk.download('punkt')

# Beispiel-Liste von Strings


# Stopwords definieren
stop_words = set(stopwords.words('english')+["like", "oh", "na", "la", "yo", "get"])

# Preprocessing: Tokenisieren, Stopwords entfernen, Kleinbuchstaben
def preprocess(doc):
    doc = re.sub(r"[^a-zA-Z0-9\s]", " ", doc) # delete unncessary things
    #doc = remove_stopwords(doc.lower())
    doc = remove_repeated_lines(doc)
    
    tokens = word_tokenize(doc.lower())  
    
    tokens = [word for word in tokens if word not in stop_words] 
    
    return tokens

def remove_repeated_lines(doc):
    lines = doc.split("\n")
    return "\n".join(sorted(set(lines), key=lines.index))

def sort_out_bad_words(doc,word_list, replacement="swear"):
    # Erstelle ein Set für einen schnelleren Lookup
    word_set = set(word_list)
    
    # Splitte den Text in Wörter
    words = doc.split()
    
    # Ersetze jedes Wort, falls es in der Liste vorkommt
    replaced_words = [replacement if word.lower() in word_set else word for word in words]
    
    # Füge den Text wieder zusammen
    return " ".join(replaced_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/maurice/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/maurice/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import csv

def processBertTopics(dataset,sentenceTransformer):
    # Initialize your topic model
    #embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
    
    embedding_model = SentenceTransformer(sentenceTransformer)
    topic_model = BERTopic(embedding_model=embedding_model)
    # Fit and transform the text data

    without_swear = [sort_out_bad_words(doc, swear_words ) for doc in dataset]
    topics, probs = topic_model.fit_transform(without_swear)

    #print(topic_model.get_topic_info())
    #print(type(topic_model.get_topics()))
    return topic_model.get_topics()

# Print or analyze the results
#print("Topics:", topics)
#print("Probabilities:", probs)

In [15]:
import time
def run_analytics_bert_different_Sentence_Transformers(amount_per_iteration,name_of_run, sentence_transformer):

    start_time = time.time()
    amount = amount_per_iteration

    rap_70_lyrics_bert = processBertTopics(subdataset_70['lyrics'][:amount],sentence_transformer)
    rap_80_lyrics_bert = processBertTopics(subdataset_80['lyrics'][:amount],sentence_transformer)
    rap_90_lyrics_bert = processBertTopics(subdataset_90['lyrics'][:amount],sentence_transformer)
    rap_00_lyrics_bert = processBertTopics(subdataset_00['lyrics'][:amount],sentence_transformer)
    rap_10_lyrics_bert = processBertTopics(subdataset_10['lyrics'][:amount],sentence_transformer)
    rap_20_lyrics_bert = processBertTopics(subdataset_20['lyrics'][:amount],sentence_transformer)
    rap_30_lyrics_bert = processBertTopics(subdataset_30['lyrics'][:amount],sentence_transformer)

    list_of_lyrics_bert = {
        "70s": rap_70_lyrics_bert,
        "80s": rap_80_lyrics_bert,
        "90s": rap_90_lyrics_bert,
        "00s": rap_00_lyrics_bert,
        "10s": rap_10_lyrics_bert,
        "20s": rap_20_lyrics_bert,
        "30s": rap_30_lyrics_bert
    }
    end_time = time.time()

    amount_of_time = end_time - start_time
    
    data_for_csv_bert = []
    
    for decade_name, decade in list_of_lyrics_bert.items():
        i = 0
        for topics in decade:
            i = i + 1
            data_for_csv_bert.append({
                "Method": "BERT",
                "Genre": "Rap",
                "Decade": decade_name,
                "Topic": str(decade[topics])
            })
    data_for_csv_bert.append({
                "Method": "TIME",
                "Genre": "TIME",
                "Decade": "TIME",
                "Topic": str(amount_of_time)
            })
    
    # Datei schreiben
    name_of_file = "topics_rap_BERT_" + str(sentence_transformer) + str(name_of_run) + ".csv"
    output_file = os.path.join("Analysis_Folder/BERT/", name_of_file)
    #output_file = "Analysis_Folder/BERT/topics_rap_BERT_.csv"
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["Method", "Genre", "Decade", "Topic"])
        writer.writeheader()  # Schreibe die Kopfzeile
        writer.writerows(data_for_csv_bert)  # Schreibe die Daten
    
    print(f"CSV-Datei '{output_file}' wurde erfolgreich erstellt!")

In [24]:
run_analytics_bert_different_Sentence_Transformers(1000,"1000Songs","all-Mpnet-base-v2")

CSV-Datei 'Analysis_Folder/BERT/topics_rap_BERT_all-Mpnet-base-v21000Songs.csv' wurde erfolgreich erstellt!
